# 출/퇴근 시간 붐비는 역에 대한 분석

## 분석을 위해 필요한 데이터

1. 출근시간 07~09 시 사이에 하차 하는 역별 이용객수 정보
2. 퇴근시간 18~20 시 사이에 승차 하는 역별 이용객수 정보
3. 토요일과 일요일은 제외해야 함

In [1]:
import pandas as pd

In [2]:
#승하차인원 csv파일 읽기
data= pd.read_csv('./data2/부산교통공사 시간대별 승하차인원 (2018년).csv',engine='python')  #한글파일 csv인 경우 engin옵션 필요

In [3]:
#컬럼 일괄 변경 
data.columns = ['년월일', '역번호', '역명', '구분', '1시', '2시', '3시', '4시', '5시','6시', '7시', '8시', '9시', '10시', '11시', '12시','13시', '14시', '15시', '16시', '17시', '18시','19시', '20시', '21시', '22시', '23시', '24시']

data.columns

Index(['년월일', '역번호', '역명', '구분', '1시', '2시', '3시', '4시', '5시', '6시', '7시',
       '8시', '9시', '10시', '11시', '12시', '13시', '14시', '15시', '16시', '17시',
       '18시', '19시', '20시', '21시', '22시', '23시', '24시'],
      dtype='object')

In [4]:
data.head()

,년월일,역번호,역명,구분,1시,2시,3시,4시,5시,6시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,20180101,95,다대포해수욕장,승차,0,0,0,12,37,63,...,430,361,398,221,129,110,87,59,12,6
1,20180101,95,다대포해수욕장,하차,12,7,0,1,53,1392,...,286,294,319,182,179,139,134,89,76,48
2,20180101,96,다대포항,승차,1,1,0,4,44,70,...,182,179,165,129,90,63,54,27,11,2
3,20180101,96,다대포항,하차,11,1,0,0,11,62,...,186,193,191,198,155,143,128,115,68,42
4,20180101,97,낫개,승차,1,1,0,9,78,159,...,248,274,250,171,146,61,78,65,22,1


In [5]:
#datatype변경 년월일:to_datetime  역번호: astype('str')
data['년월일']=data['년월일'].astype('str')  # int -> str변환후 날짜 변환해야 함
data['년월일']=pd.to_datetime(data['년월일'])
data['역번호']=data['역번호'].astype('str')

In [6]:
data.tail()

,년월일,역번호,역명,구분,1시,2시,3시,4시,5시,6시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
81755,2018-12-31,412,동부산대,하차,16,3,0,2,3,37,...,239,272,308,266,185,117,95,89,85,50
81756,2018-12-31,413,고 촌,승차,0,0,0,3,16,23,...,49,46,55,37,17,14,14,16,9,0
81757,2018-12-31,413,고 촌,하차,0,1,0,0,0,23,...,43,33,43,53,35,20,17,16,8,4
81758,2018-12-31,414,안 평,승차,0,0,0,2,34,55,...,144,148,153,159,63,43,55,25,9,3
81759,2018-12-31,414,안 평,하차,7,0,0,0,3,46,...,124,172,147,222,126,77,75,64,42,23


### 데이터 정제 작업

In [7]:
##역명의 양쪽공백 문자 제거 
data['역명'] = data['역명'].str.strip()
##역명의 중간 공백 문자 제거
data['역명'] = data['역명'].str.replace(" ","")

In [8]:
data.head()

,년월일,역번호,역명,구분,1시,2시,3시,4시,5시,6시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,2018-01-01,95,다대포해수욕장,승차,0,0,0,12,37,63,...,430,361,398,221,129,110,87,59,12,6
1,2018-01-01,95,다대포해수욕장,하차,12,7,0,1,53,1392,...,286,294,319,182,179,139,134,89,76,48
2,2018-01-01,96,다대포항,승차,1,1,0,4,44,70,...,182,179,165,129,90,63,54,27,11,2
3,2018-01-01,96,다대포항,하차,11,1,0,0,11,62,...,186,193,191,198,155,143,128,115,68,42
4,2018-01-01,97,낫개,승차,1,1,0,9,78,159,...,248,274,250,171,146,61,78,65,22,1


In [9]:
#데이터 검증 역별로 데이터건수가 동일한지 체크해보자
#df.groupby(['년월일','역번호','역명'])['합계'].mean()
d_check =pd.DataFrame(data.groupby(['역번호','역명','구분'])['년월일'].count())
d_check=d_check.reset_index()
d_check.head()
#년월일 갯수가 365가 아닌 역 체크해보자


,역번호,역명,구분,년월일
0,100,동매,승차,365
1,100,동매,하차,365
2,101,신평,승차,365
3,101,신평,하차,365
4,102,하단,승차,365


In [10]:
#역번호가 같은경우 역통합이 필요함 : 
#서면 -> 1서면 : 119
#연산 -> 1연산 : 123
#동래 -> 1동래 : 125
#서면 -> 2서면 : 219
#덕천 -> 2덕천 : 233
#연산 -> 3연산 : 305
#덕천 -> 3덕천 : 313
#동래 -> 4동래 : 402

#역번호 오류 추정 수안역
#7 -> 403  : 수안역

d_check[d_check['년월일']<365]

,역번호,역명,구분,년월일
38,119,1서면,승차,153
39,119,1서면,하차,153
40,119,서면,승차,212
41,119,서면,하차,212
48,123,1연산,승차,153
49,123,1연산,하차,153
50,123,연산,승차,212
51,123,연산,하차,212
54,125,1동래,승차,153
55,125,1동래,하차,153


In [11]:
len(data[data['역번호']=='119']) #730

730

In [12]:
# 역번호 119 이면서 역이름 서면 인건 --> 1서면으로 변경 필요 데이터  :424
len(data[(data['역번호']=='119') & (data['역명']=='서면')])

424

In [13]:
# apply() 함수로 데이터를 변경해볼까?
# 역번호 역명 기준으로 가져온  수정할대상을 가져와서 원하는 역명으로 수정  전체 데이터 작업이라 오래 걸린다.
data['역명'] = data.apply(lambda x : '1서면' if (x.역번호=='119') & (x.역명=='서면') else x.역명  , axis=1 )

In [14]:
data[data['역번호']=='119']

,년월일,역번호,역명,구분,1시,2시,3시,4시,5시,6시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
48,2018-01-01,119,1서면,승차,7,0,0,22,1053,1070,...,2177,2383,2429,2255,1926,1781,1754,1582,1076,16
49,2018-01-01,119,1서면,하차,54,0,0,1,154,192,...,2614,2402,2494,2026,1455,1059,772,610,348,157
272,2018-01-02,119,1서면,승차,4,0,0,10,362,432,...,3372,3846,3984,4462,3018,2576,2413,2274,1502,22
273,2018-01-02,119,1서면,하차,0,0,0,1,159,313,...,3465,3463,3737,5024,2498,1237,873,621,353,79
496,2018-01-03,119,1서면,승차,0,0,0,5,383,421,...,3176,3792,3963,4570,2844,2729,2572,2465,1533,10
497,2018-01-03,119,1서면,하차,0,0,0,0,179,311,...,3348,3198,3815,5362,2711,1191,964,659,386,95
720,2018-01-04,119,1서면,승차,0,0,0,4,367,435,...,3248,3782,4166,4496,3034,2773,2684,2583,1682,11
721,2018-01-04,119,1서면,하차,0,0,0,0,168,312,...,3468,3379,4058,5592,2758,1263,1043,635,379,68
944,2018-01-05,119,1서면,승차,0,0,0,6,344,367,...,3385,3917,4266,4779,3244,2654,3061,2839,2329,19
945,2018-01-05,119,1서면,하차,0,0,0,0,160,292,...,3709,3648,4337,6685,3903,1853,1490,919,599,131


In [15]:
data[(data['역번호']=='119') & (data['역명']=='서면')]

,년월일,역번호,역명,구분,1시,2시,3시,4시,5시,6시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시


In [16]:
#데이터 변경후 재검증 
d_check = pd.DataFrame(data.groupby(['역번호','역명','구분'])['년월일'].count())
d_check = d_check.reset_index()
#년월일 갯수가 365가 아닌 역 체크해보자
d_check[d_check['년월일']<365]

,역번호,역명,구분,년월일
46,123,1연산,승차,153
47,123,1연산,하차,153
48,123,연산,승차,212
49,123,연산,하차,212
52,125,1동래,승차,153
53,125,1동래,하차,153
54,125,동래,승차,212
55,125,동래,하차,212
110,219,2서면,승차,153
111,219,2서면,하차,153


In [17]:
# 어차피 여러건 처리해야 한다면 lambda 대신 함수를 정의하고 해볼까?
#data['역명'] = data.apply(lambda x : '1서면' if (x.역번호=='119') & (x.역명=='서면') else x.역명  , axis=1 )
#역번호가 같은경우 역통합이 필요함 : 
#서면 -> 1서면 : 119
#연산 -> 1연산 : 123
#동래 -> 1동래 : 125
#서면 -> 2서면 : 219
#덕천 -> 2덕천 : 233
#연산 -> 3연산 : 305
#덕천 -> 3덕천 : 313
#동래 -> 4동래 : 402

def dfilter(x):
    if   (x.역번호=='123') & (x.역명=='연산') : return '1연산'
    elif (x.역번호=='125') & (x.역명=='동래') : return '1동래'
    elif (x.역번호=='219') & (x.역명=='서면') : return '2서면'
    elif (x.역번호=='233') & (x.역명=='덕천') : return '2덕천'
    elif (x.역번호=='305') & (x.역명=='연산') : return '3연산'
    elif (x.역번호=='313') & (x.역명=='덕천') : return '3덕천'
    elif (x.역번호=='402') & (x.역명=='동래') : return '4동래'
    else: return x.역명

# data['역명'] = data.apply(dfilter)
data.head()

#근데 잘 안된다... 함수로 정의한게 시리즈여서 그런가? axis=1 옵션이 안들어가서 인가?
#일단 람다로 여러번 하자.

,년월일,역번호,역명,구분,1시,2시,3시,4시,5시,6시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,2018-01-01,95,다대포해수욕장,승차,0,0,0,12,37,63,...,430,361,398,221,129,110,87,59,12,6
1,2018-01-01,95,다대포해수욕장,하차,12,7,0,1,53,1392,...,286,294,319,182,179,139,134,89,76,48
2,2018-01-01,96,다대포항,승차,1,1,0,4,44,70,...,182,179,165,129,90,63,54,27,11,2
3,2018-01-01,96,다대포항,하차,11,1,0,0,11,62,...,186,193,191,198,155,143,128,115,68,42
4,2018-01-01,97,낫개,승차,1,1,0,9,78,159,...,248,274,250,171,146,61,78,65,22,1


In [18]:
data['역명'] = data.apply(lambda x : '1연산' if (x.역번호=='123') & (x.역명=='연산') else x.역명  , axis=1 )

In [19]:
data['역명'] = data.apply(lambda x : '1동래' if (x.역번호=='125') & (x.역명=='동래') else x.역명  , axis=1 )

In [20]:
data['역명'] = data.apply(lambda x : '2서면' if (x.역번호=='219') & (x.역명=='서면') else x.역명  , axis=1 )

In [21]:
data['역명'] = data.apply(lambda x : '2덕천' if (x.역번호=='233') & (x.역명=='덕천') else x.역명  , axis=1 )


In [22]:
data['역명'] = data.apply(lambda x : '3연산' if (x.역번호=='305') & (x.역명=='연산') else x.역명  , axis=1 )

In [23]:
data['역명'] = data.apply(lambda x : '3덕천' if (x.역번호=='313') & (x.역명=='덕천') else x.역명  , axis=1 )

In [24]:
data['역명'] = data.apply(lambda x : '4동래' if (x.역번호=='402') & (x.역명=='동래') else x.역명  , axis=1 )

In [25]:
#데이터 변경후 재검증 
d_check = pd.DataFrame(data.groupby(['역번호','역명','구분'])['년월일'].count())
d_check = d_check.reset_index()
#년월일 갯수가 365가 아닌 역 체크해보자
d_check[d_check['년월일']<365]

,역번호,역명,구분,년월일
190,403,수안,승차,362
214,7,수안,승차,3


In [26]:
#역번호 오류 추정 수안역
#7 -> 403  : 수안역
data['역번호'] = data.apply(lambda x : '403' if (x.역번호=='7') & (x.역명=='수안') else x.역번호  , axis=1 )

In [27]:
data[data['역번호']=='403']

,년월일,역번호,역명,구분,1시,2시,3시,4시,5시,6시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
200,2018-01-01,403,수안,승차,2,0,0,0,30,47,...,192,206,184,142,108,72,47,43,21,8
201,2018-01-01,403,수안,하차,2,0,0,0,12,42,...,141,165,169,99,53,38,44,35,25,1
424,2018-01-02,403,수안,승차,0,0,0,0,35,85,...,365,367,368,451,207,117,97,76,46,7
425,2018-01-02,403,수안,하차,0,0,0,0,18,57,...,316,259,296,353,140,92,49,35,30,10
648,2018-01-03,403,수안,승차,0,0,0,1,35,77,...,354,328,390,415,212,156,109,70,37,5
649,2018-01-03,403,수안,하차,0,0,0,0,16,56,...,337,248,316,369,180,90,55,41,17,3
872,2018-01-04,403,수안,승차,0,0,0,0,38,81,...,323,392,349,402,222,114,107,93,42,8
873,2018-01-04,403,수안,하차,0,0,0,0,20,58,...,302,254,320,393,167,80,50,36,29,4
1096,2018-01-05,403,수안,승차,0,0,0,0,34,69,...,374,391,377,417,243,132,109,97,53,9
1097,2018-01-05,403,수안,하차,0,0,0,0,21,51,...,311,267,370,414,174,91,53,45,41,2


In [28]:
#데이터 변경후 재검증 
d_check = pd.DataFrame(data.groupby(['역번호','역명','구분'])['년월일'].count())
d_check = d_check.reset_index()
#년월일 갯수가 365가 아닌 역 체크해보자
d_check[d_check['년월일']<365]

,역번호,역명,구분,년월일


In [29]:
d_check

,역번호,역명,구분,년월일
0,100,동매,승차,365
1,100,동매,하차,365
2,101,신평,승차,365
3,101,신평,하차,365
4,102,하단,승차,365
5,102,하단,하차,365
6,103,당리,승차,365
7,103,당리,하차,365
8,104,사하,승차,365
9,104,사하,하차,365


In [30]:
data.head()

,년월일,역번호,역명,구분,1시,2시,3시,4시,5시,6시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,2018-01-01,95,다대포해수욕장,승차,0,0,0,12,37,63,...,430,361,398,221,129,110,87,59,12,6
1,2018-01-01,95,다대포해수욕장,하차,12,7,0,1,53,1392,...,286,294,319,182,179,139,134,89,76,48
2,2018-01-01,96,다대포항,승차,1,1,0,4,44,70,...,182,179,165,129,90,63,54,27,11,2
3,2018-01-01,96,다대포항,하차,11,1,0,0,11,62,...,186,193,191,198,155,143,128,115,68,42
4,2018-01-01,97,낫개,승차,1,1,0,9,78,159,...,248,274,250,171,146,61,78,65,22,1


In [31]:
data.tail()

,년월일,역번호,역명,구분,1시,2시,3시,4시,5시,6시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
81755,2018-12-31,412,동부산대,하차,16,3,0,2,3,37,...,239,272,308,266,185,117,95,89,85,50
81756,2018-12-31,413,고촌,승차,0,0,0,3,16,23,...,49,46,55,37,17,14,14,16,9,0
81757,2018-12-31,413,고촌,하차,0,1,0,0,0,23,...,43,33,43,53,35,20,17,16,8,4
81758,2018-12-31,414,안평,승차,0,0,0,2,34,55,...,144,148,153,159,63,43,55,25,9,3
81759,2018-12-31,414,안평,하차,7,0,0,0,3,46,...,124,172,147,222,126,77,75,64,42,23


In [32]:
#정제된 데이터 파일로 저장 :excel은 근데 너무 오래 걸림
data.to_excel('./data2/부산일자별도시철도이용객수_v1.xlsx')

In [37]:
#정제된 데이터 파일로 저장 : csv로 빠르게 인덱스 없이! 한글깨짐 현상 때문에 encoding필요 
data.to_csv('./data2/부산일자별도시철도이용객수_v1.csv',encoding='ms949',index=False)

## 여기까지 데이터 정제, 출/퇴근 데이터는 v2부터